# Definitive Version

## Gather all the data

In [3]:
import fastf1
import pandas as pd
import os

In [8]:
year = 2023
#Get the schedule for the year
year_schedule = fastf1.get_event_schedule(year)
print(year_schedule['RoundNumber'])

#Repeat for every race 
for round in year_schedule['RoundNumber'][1:]: #Ignore the first round because it's the testing event of the season, not an actual race

    #Create a dircetory where to store all the files
    dirname = '../data/'+str(round)+'_'+year_schedule['Location'][round]
    os.mkdir(dirname)

    #Get the race and load the data
    race = fastf1.get_session(year, year_schedule['Location'][round], 'R')
    race.load()

    #Laps
    race.laps.to_csv(dirname+'/laps.csv')
    #Results
    race.results.to_csv(dirname+'/results.csv')
    #Race info (unused)
    race.event.to_csv(dirname+'/race_info.csv')
    #Weahter data (unused)
    race.weather_data.to_csv(dirname+'/weather_data.csv')
    #Track status
    race.track_status.to_csv(dirname+'/track_status.csv')
    
    #Create a subdirectory for the car telemetries
    for i in race.car_data.keys():
        race.car_data[i].to_csv(dirname+'/'+str(i)+'_telemetry.csv')


core           INFO 	Loading data for Bahrain Grand Prix - Race [v3.2.0]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info


req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for lap_count
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...


0      0
1      1
2      2
3      3
4      4
5      5
6      6
7      7
8      8
9      9
10    10
11    11
12    12
13    13
14    14
15    15
16    16
17    17
18    18
19    19
20    20
21    21
22    22
Name: RoundNumber, dtype: int32


req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['1', '11', '14', '55', '44', '18', '63', '77', '10', '23', '22', '2', '20', '21', '27', '24', '4', '31', '16', '81']
core           INFO 	Loading data for Saudi Arabian Grand Prix - Race [v3.2.0]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for lap_count
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
req            INFO 	Using cached data for car_data
req

# Data Aggregation and tSNE

In [21]:
import pandas as pd
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans
from sklearn.discriminant_analysis import StandardScaler
import fastf1

In [37]:

year = 2023
year_schedule = fastf1.get_event_schedule(year)
for round in year_schedule['RoundNumber'][1:]:
    race = fastf1.get_session(year, year_schedule['Location'][round], 'R')
    race.load()
    dataset = race.laps

     # Aggregate additional data to the dataset
    df_to_append = pd.DataFrame(columns=['avg_speed', 'max_speed', 'min_speed', 'avg_throttle', 
                                     'full_throttle', 'full_brake', 'air_temp', 'track_temp', 
                                     'wind_dir', 'wind_speed'])
    
    for index in range(len(dataset)):
        lap = (dataset.iloc[index])
        # get the telemetry data for the lap via convenient method
        telemetry = lap.get_car_data()
        # compute some metrics from the telemetry data
        # avg speed over the lap
        avg_speed = telemetry['Speed'].mean()
        # max speed over the lap
        max_speed = telemetry['Speed'].max()
        # min speed over the lap
        min_speed = telemetry['Speed'].min()
        # avg throttle % over the lap
        avg_throttle = telemetry['Throttle'].mean()
        # % of the lap spent at full throttle
        full_throttle = telemetry[telemetry['Throttle'] == 100].shape[0] / telemetry.shape[0]
        # % of the lap spent breaking
        full_brake = telemetry[telemetry['Brake'] == True].shape[0] / telemetry.shape[0]
    
        # get the weather data for the lap
        weather = lap.get_weather_data()
        # get some data from the weather data
        # air temperature
        air_temp = weather['AirTemp']
        # track temperature
        track_temp = weather['TrackTemp']
        # wind direction
        wind_dir = weather['WindDirection']
        # wind speed
        wind_speed = weather['WindSpeed']

        # create a new dataframe with the new data
        df_to_append.loc[index] = [avg_speed, max_speed, min_speed, avg_throttle, full_throttle, full_brake, air_temp, track_temp, wind_dir, wind_speed]

    #add the new data to the current lap data
    dataset = pd.concat([dataset, df_to_append], axis=1)
    print('Augmented Dataset: \n', dataset)

    # Clean up the dataset before dimensionality reduction
    # Remove rows that correspond to laps where the driver pitted, beacause they are obvious outliers
    rows_to_remove = dataset.index[dataset['PitInTime'].notnull()].tolist()
    dataset = dataset.drop(rows_to_remove, axis = 0)
    rows_to_remove = dataset.index[dataset['PitOutTime'].notnull()].tolist()
    dataset = dataset.drop(rows_to_remove, axis = 0)
    
    cols_to_remove =['Time', 'DriverNumber','PitOutTime','PitInTime', 'Sector1SessionTime', 
                        'Sector2SessionTime', 'Sector3SessionTime', 'SpeedI1', 'SpeedI2',
                        'SpeedFL', 'SpeedST', 'IsPersonalBest',
                        'FreshTyre','LapStartTime','LapStartDate', 'Deleted',
                        'DeletedReason','FastF1Generated','IsAccurate']
    
    dataset = dataset.drop(cols_to_remove, axis = 1)
    dataset['TrackStatus'] = dataset['TrackStatus'].astype('float') #I hate this
    #drop rows where track status is not 1, i.e. the lap is not a 'racing' lap
    rows_to_remove = dataset.index[dataset['TrackStatus'] != 1].tolist()
    dataset = dataset.drop(rows_to_remove, axis = 0)
    #drop rows with missing data because i cant think of something better
    dataset = dataset.dropna(axis=0)
    #drop trackStatus as well after i used i to remove the rows i wanted
    dataset = dataset.drop('TrackStatus', axis = 1)
    #Convert timedelta64 to seconds
    cols_to_change = ['LapTime','Sector1Time',
          'Sector2Time', 'Sector3Time']
    for col in cols_to_change:
        dataset[col] = dataset[col].dt.total_seconds()
    dataset = dataset.reset_index()
    print(dataset.info())

    # TSNE
    cols = ['LapTime', 'Sector1Time','Sector2Time', 
            'Sector3Time', 'TyreLife', 'avg_speed', 
            'max_speed', 'min_speed', 'avg_throttle', 
            'full_throttle', 'full_brake', 'air_temp', 
            'track_temp', 'wind_dir', 'wind_speed']
    scaler= StandardScaler()
    scaled = scaler.fit_transform(dataset[cols])
    print(scaled)
    tsne = TSNE(n_components=2, perplexity=30, learning_rate = 500, n_iter=5000,
                n_iter_without_progress=500, random_state=16)
    tsne_results = tsne.fit_transform(scaled)
    tsne_df = pd.DataFrame(data = tsne_results, columns=['dim1','dim2'])
    print(tsne_df)
    
    #KMeans
    k_means_tsne = KMeans(n_clusters=3, n_init=100, max_iter=400, init="k-means++", random_state=16).fit(tsne_df)
    labels_tsne = k_means_tsne.labels_
    clusters_tsne = pd.concat([tsne_df, pd.DataFrame({'tsne_clusters':labels_tsne})], axis=1)
    clusters_tsne.columns=['dim1','dim2','tsne_clusters_id']
    print(clusters_tsne)
    print(clusters_tsne['tsne_clusters_id'].value_counts())

    result_df = pd.concat([dataset, clusters_tsne], axis=1)
    print(result_df)

    dirname = '../data/'+str(round)+'_'+year_schedule['Location'][round]
    result_df.to_csv(dirname+'/'+'tSNE.csv')
    

core           INFO 	Loading data for Bahrain Grand Prix - Race [v3.2.0]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for lap_count
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['1', '11', '14', '55', '44', '18', '63', '77', '10', '23', '22', '2', '20', '21', '27', '24', '4', '31', '16', '81']


Augmented Dataset: 
                        Time Driver DriverNumber                LapTime  \
0    0 days 01:04:15.961000    VER            1 0 days 00:01:39.019000   
1    0 days 01:05:53.935000    VER            1 0 days 00:01:37.974000   
2    0 days 01:07:31.941000    VER            1 0 days 00:01:38.006000   
3    0 days 01:09:09.917000    VER            1 0 days 00:01:37.976000   
4    0 days 01:10:47.952000    VER            1 0 days 00:01:38.035000   
...                     ...    ...          ...                    ...   
1051 0 days 01:17:53.364000    PIA           81 0 days 00:01:41.534000   
1052 0 days 01:19:34.948000    PIA           81 0 days 00:01:41.584000   
1053 0 days 01:21:16.300000    PIA           81 0 days 00:01:41.352000   
1054 0 days 01:22:57.456000    PIA           81 0 days 00:01:41.156000   
1055 0 days 01:24:59.527000    PIA           81 0 days 00:02:02.071000   

      LapNumber  Stint PitOutTime              PitInTime  \
0           1.0    1.0        

core           INFO 	Loading data for Saudi Arabian Grand Prix - Race [v3.2.0]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info


          dim1       dim2  tsne_clusters_id
0     3.029480  21.649624                 1
1     3.424211  21.277430                 1
2     3.732482  23.450529                 1
3     4.536261  22.262226                 1
4     5.085627  22.198530                 1
..         ...        ...               ...
884  13.423263  55.614983                 1
885  13.613481  53.920120                 1
886  13.042974  54.456848                 1
887  12.396272  53.446381                 1
888  12.183684  48.053638                 1

[889 rows x 3 columns]
tsne_clusters_id
1    311
2    295
0    283
Name: count, dtype: int64
     index Driver  LapTime  LapNumber  Stint  Sector1Time  Sector2Time  \
0        2    VER   98.006        3.0    1.0       31.388       42.469   
1        3    VER   97.976        4.0    1.0       31.271       42.642   
2        4    VER   98.035        5.0    1.0       31.244       42.724   
3        5    VER   97.986        6.0    1.0       31.341       42.632   
4       

req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for lap_count
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['11', '1', '14', '63', '44', '55', '16', '31', '10', '20', '22', '27', '24', '21', '81', '2', '4', '77', '23', '18']


...

Augmented Dataset: 
                       Time Driver DriverNumber                LapTime  \
0   0 days 00:15:48.822000    VER            1 0 days 00:01:45.670000   
1   0 days 00:18:01.833000    VER            1 0 days 00:02:13.011000   
2   0 days 00:19:51.821000    VER            1 0 days 00:01:49.988000   
3   0 days 00:22:57.617000    VER            1                    NaT   
4   0 days 00:25:45.267000    VER            1                    NaT   
..                     ...    ...          ...                    ...   
941 0 days 01:37:17.677000    PIA           81 0 days 00:01:35.585000   
942 0 days 01:38:53.167000    PIA           81 0 days 00:01:35.490000   
943 0 days 01:40:28.762000    PIA           81 0 days 00:01:35.595000   
944 0 days 01:42:04.776000    PIA           81 0 days 00:01:36.014000   
945 0 days 01:43:40.904000    PIA           81 0 days 00:01:36.128000   

     LapNumber  Stint PitOutTime PitInTime            Sector1Time  \
0          1.0    1.0        NaT 

core           INFO 	Loading data for Abu Dhabi Grand Prix - Race [v3.2.0]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info


          dim1       dim2  tsne_clusters_id
0   -29.181570  -5.343487                 1
1   -27.357248  -5.422328                 1
2   -26.742758  -5.300110                 1
3   -26.218601  -4.721538                 1
4   -10.231490  -2.035070                 1
..         ...        ...               ...
639  34.798832  -4.441444                 2
640  26.359085  10.596945                 0
641  31.770891  14.414827                 0
642  31.464724  14.283478                 0
643  31.247698  13.672464                 0

[644 rows x 3 columns]
tsne_clusters_id
1    268
0    222
2    154
Name: count, dtype: int64
     index Driver  LapTime  LapNumber  Stint  Sector1Time  Sector2Time  \
0        6    VER  100.785        7.0    1.0       29.256       33.440   
1        7    VER  100.142        8.0    1.0       29.033       33.306   
2        8    VER   99.669        9.0    1.0       28.653       33.163   
3        9    VER   99.484       10.0    1.0       28.549       33.175   
4       

req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for lap_count
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['1', '16', '63', '11', '4', '81', '14', '22', '44', '18', '3', '31', '10', '23', '27', '2', '24', '55', '77', '20']


Augmented Dataset: 
                        Time Driver DriverNumber                LapTime  \
0    0 days 01:03:57.047000    VER            1 0 days 00:01:32.190000   
1    0 days 01:05:27.757000    VER            1 0 days 00:01:30.710000   
2    0 days 01:06:58.165000    VER            1 0 days 00:01:30.408000   
3    0 days 01:08:28.881000    VER            1 0 days 00:01:30.716000   
4    0 days 01:09:59.433000    VER            1 0 days 00:01:30.552000   
...                     ...    ...          ...                    ...   
1152 0 days 02:23:40.213000    MAG           20 0 days 00:01:30.445000   
1153 0 days 02:25:10.420000    MAG           20 0 days 00:01:30.207000   
1154 0 days 02:26:40.672000    MAG           20 0 days 00:01:30.252000   
1155 0 days 02:28:11.101000    MAG           20 0 days 00:01:30.429000   
1156 0 days 02:29:42.357000    MAG           20 0 days 00:01:31.256000   

      LapNumber  Stint PitOutTime PitInTime            Sector1Time  \
0           1.0    1

# Data Aggregation and PCA

In [1]:
import fastf1
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import StandardScaler

In [2]:

year = 2023
year_schedule = fastf1.get_event_schedule(year)
for round in year_schedule['RoundNumber'][1:]:
    race = fastf1.get_session(year, year_schedule['Location'][round], 'R')
    race.load()
    dataset = race.laps

     # Aggregate additional data to the dataset
    df_to_append = pd.DataFrame(columns=['avg_speed', 'max_speed', 'min_speed', 'avg_throttle', 
                                     'full_throttle', 'full_brake', 'air_temp', 'track_temp', 
                                     'wind_dir', 'wind_speed'])
    
    for index in range(len(dataset)):
        lap = (dataset.iloc[index])
        # get the telemetry data for the lap via convenient method
        telemetry = lap.get_car_data()
        # compute some metrics from the telemetry data
        # avg speed over the lap
        avg_speed = telemetry['Speed'].mean()
        # max speed over the lap
        max_speed = telemetry['Speed'].max()
        # min speed over the lap
        min_speed = telemetry['Speed'].min()
        # avg throttle % over the lap
        avg_throttle = telemetry['Throttle'].mean()
        # % of the lap spent at full throttle
        full_throttle = telemetry[telemetry['Throttle'] == 100].shape[0] / telemetry.shape[0]
        # % of the lap spent breaking
        full_brake = telemetry[telemetry['Brake'] == True].shape[0] / telemetry.shape[0]
    
        # get the weather data for the lap
        weather = lap.get_weather_data()
        # get some data from the weather data
        # air temperature
        air_temp = weather['AirTemp']
        # track temperature
        track_temp = weather['TrackTemp']
        # wind direction
        wind_dir = weather['WindDirection']
        # wind speed
        wind_speed = weather['WindSpeed']

        # create a new dataframe with the new data
        df_to_append.loc[index] = [avg_speed, max_speed, min_speed, avg_throttle, full_throttle, full_brake, air_temp, track_temp, wind_dir, wind_speed]

    #add the new data to the current lap data
    dataset = pd.concat([dataset, df_to_append], axis=1)
    print('after: \n', dataset)

    # Clean up the dataset before dimensionality reduction
    # Remove rows that correspond to laps where the driver pitted, beacause they are obvious outliers
    rows_to_remove = dataset.index[dataset['PitInTime'].notnull()].tolist()
    dataset = dataset.drop(rows_to_remove, axis = 0)
    rows_to_remove = dataset.index[dataset['PitOutTime'].notnull()].tolist()
    dataset = dataset.drop(rows_to_remove, axis = 0)
    
    cols_to_remove =['Time', 'DriverNumber','PitOutTime','PitInTime', 'Sector1SessionTime', 
                        'Sector2SessionTime', 'Sector3SessionTime', 'SpeedI1', 'SpeedI2',
            'SpeedFL', 'SpeedST', 'IsPersonalBest',
                        'FreshTyre','LapStartTime','LapStartDate', 'Deleted',
                        'DeletedReason','FastF1Generated','IsAccurate']
    dataset = dataset.drop(cols_to_remove, axis = 1)
    dataset['TrackStatus'] = dataset['TrackStatus'].astype('float') #I hate this
    #drop rows where track status is not 1, i.e. the lap is not a 'racing' lap
    rows_to_remove = dataset.index[dataset['TrackStatus'] != 1].tolist()
    dataset = dataset.drop(rows_to_remove, axis = 0)
    #drop rows with missing data because i cant think of something better
    dataset = dataset.dropna(axis=0)
    #drop trackStatus as well after i used i to remove the rows i wanted
    dataset = dataset.drop('TrackStatus', axis = 1)
    #Convert timedelta64 to seconds
    cols_to_change = ['LapTime','Sector1Time',
          'Sector2Time', 'Sector3Time']
    for col in cols_to_change:
        dataset[col] = dataset[col].dt.total_seconds()
    dataset = dataset.reset_index()
    print(dataset.info())

    # PCA
    cols = ['Sector1Time','Sector2Time', 
            'Sector3Time',  'TyreLife', 'avg_speed', 
            'max_speed', 'min_speed', 'avg_throttle', 
            'full_throttle', 'full_brake', 'air_temp', 
            'track_temp', 'wind_dir', 'wind_speed']

    scaler= StandardScaler()
    scaled = scaler.fit_transform(dataset[cols])

    pca = PCA(n_components = 2, random_state=16)
    components = pca.fit_transform(scaled)
    pca_df = pd.DataFrame(data = components, columns = ['dim1','dim2'])

    #KMeans
    kmeans = KMeans(n_clusters = 3)
    clusters = kmeans.fit(pca_df)
    labels = kmeans.labels_
    clusters = pd.concat([pca_df, pd.DataFrame({'kmeans_clusters':labels})], axis=1)
    clusters.columns=['dim1','dim2','cluster_id']
    result_df = pd.concat([dataset, clusters], axis=1)
       
    print(f'\n\n\n {year_schedule["Location"][round]} results_df: \n', result_df)

    dirname = '../data/'+str(round)+'_'+year_schedule['Location'][round]
    result_df.to_csv(dirname+'/'+'PCA.csv')

req         WARNING 	DEFAULT CACHE ENABLED! (2.34 GB) C:\Users\perot\AppData\Local\Temp\fastf1
core           INFO 	Loading data for Bahrain Grand Prix - Race [v3.2.0]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for lap_count
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['1', '11', '14', '55', '44', '18', '63', '77', '10', '23', '22', '2', '20', '21', '

after: 
                        Time Driver DriverNumber                LapTime  \
0    0 days 01:04:15.961000    VER            1 0 days 00:01:39.019000   
1    0 days 01:05:53.935000    VER            1 0 days 00:01:37.974000   
2    0 days 01:07:31.941000    VER            1 0 days 00:01:38.006000   
3    0 days 01:09:09.917000    VER            1 0 days 00:01:37.976000   
4    0 days 01:10:47.952000    VER            1 0 days 00:01:38.035000   
...                     ...    ...          ...                    ...   
1051 0 days 01:17:53.364000    PIA           81 0 days 00:01:41.534000   
1052 0 days 01:19:34.948000    PIA           81 0 days 00:01:41.584000   
1053 0 days 01:21:16.300000    PIA           81 0 days 00:01:41.352000   
1054 0 days 01:22:57.456000    PIA           81 0 days 00:01:41.156000   
1055 0 days 01:24:59.527000    PIA           81 0 days 00:02:02.071000   

      LapNumber  Stint PitOutTime              PitInTime  \
0           1.0    1.0        NaT         

core           INFO 	Loading data for Saudi Arabian Grand Prix - Race [v3.2.0]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info





 Sakhir results_df: 
      index Driver  LapTime  LapNumber  Stint  Sector1Time  Sector2Time  \
0        2    VER   98.006        3.0    1.0       31.388       42.469   
1        3    VER   97.976        4.0    1.0       31.271       42.642   
2        4    VER   98.035        5.0    1.0       31.244       42.724   
3        5    VER   97.986        6.0    1.0       31.341       42.632   
4        6    VER   98.021        7.0    1.0       31.290       42.763   
..     ...    ...      ...        ...    ...          ...          ...   
884   1050    PIA  101.295        8.0    1.0       31.665       44.590   
885   1051    PIA  101.534        9.0    1.0       31.989       44.548   
886   1052    PIA  101.584       10.0    1.0       31.779       44.617   
887   1053    PIA  101.352       11.0    1.0       31.894       44.557   
888   1054    PIA  101.156       12.0    1.0       32.100       44.235   

     Sector3Time Compound  TyreLife  ... avg_throttle  full_throttle  \
0         24.1

req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for lap_count
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['11', '1', '14', '63', '44', '55', '16', '31', '10', '20', '22', '27', '24', '21', '81', '2', '4', '77', '23', '18']
core           INFO 	Loading data for Australian Grand Prix - Race [v3.2.0]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info


...

after: 
                        Time Driver DriverNumber                LapTime  \
0    0 days 01:04:23.001000    VER            1 0 days 00:01:59.857000   
1    0 days 01:06:46.357000    VER            1 0 days 00:02:23.356000   
2    0 days 01:32:11.479000    VER            1                    NaT   
3    0 days 01:34:24.149000    VER            1 0 days 00:02:12.670000   
4    0 days 01:35:39.894000    VER            1 0 days 00:01:15.745000   
...                     ...    ...          ...                    ...   
1104 0 days 01:56:43.133000    ZHO           24 0 days 00:01:16.767000   
1105 0 days 01:58:13.479000    ZHO           24 0 days 00:01:30.346000   
1106 0 days 01:04:23.001000    MAG           20                    NaT   
1107 0 days 01:04:23.001000    ALB           23                    NaT   
1108 0 days 01:04:23.001000    LEC           16                    NaT   

      LapNumber  Stint             PitOutTime              PitInTime  \
0           1.0    1.0        

req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for lap_count
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['1', '16', '11', '31', '18', '55', '44', '63', '14', '81', '10', '23', '20', '3', '24', '2', '77', '22', '27', '4']
core           INFO 	Loading data for Abu Dhabi Grand Prix - Race [v3.2.0]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info


after: 
                       Time Driver DriverNumber                LapTime  \
0   0 days 00:15:48.822000    VER            1 0 days 00:01:45.670000   
1   0 days 00:18:01.833000    VER            1 0 days 00:02:13.011000   
2   0 days 00:19:51.821000    VER            1 0 days 00:01:49.988000   
3   0 days 00:22:57.617000    VER            1                    NaT   
4   0 days 00:25:45.267000    VER            1                    NaT   
..                     ...    ...          ...                    ...   
941 0 days 01:37:17.677000    PIA           81 0 days 00:01:35.585000   
942 0 days 01:38:53.167000    PIA           81 0 days 00:01:35.490000   
943 0 days 01:40:28.762000    PIA           81 0 days 00:01:35.595000   
944 0 days 01:42:04.776000    PIA           81 0 days 00:01:36.014000   
945 0 days 01:43:40.904000    PIA           81 0 days 00:01:36.128000   

     LapNumber  Stint PitOutTime PitInTime            Sector1Time  \
0          1.0    1.0        NaT       NaT   

req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for lap_count
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['1', '16', '63', '11', '4', '81', '14', '22', '44', '18', '3', '31', '10', '23', '27', '2', '24', '55', '77', '20']


after: 
                        Time Driver DriverNumber                LapTime  \
0    0 days 01:03:57.047000    VER            1 0 days 00:01:32.190000   
1    0 days 01:05:27.757000    VER            1 0 days 00:01:30.710000   
2    0 days 01:06:58.165000    VER            1 0 days 00:01:30.408000   
3    0 days 01:08:28.881000    VER            1 0 days 00:01:30.716000   
4    0 days 01:09:59.433000    VER            1 0 days 00:01:30.552000   
...                     ...    ...          ...                    ...   
1152 0 days 02:23:40.213000    MAG           20 0 days 00:01:30.445000   
1153 0 days 02:25:10.420000    MAG           20 0 days 00:01:30.207000   
1154 0 days 02:26:40.672000    MAG           20 0 days 00:01:30.252000   
1155 0 days 02:28:11.101000    MAG           20 0 days 00:01:30.429000   
1156 0 days 02:29:42.357000    MAG           20 0 days 00:01:31.256000   

      LapNumber  Stint PitOutTime PitInTime            Sector1Time  \
0           1.0    1.0        Na

# Data Aggregation and MDS

In [35]:
import fastf1
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.manifold import MDS
from sklearn.discriminant_analysis import StandardScaler

In [36]:

year = 2023
year_schedule = fastf1.get_event_schedule(year)
for round in year_schedule['RoundNumber'][1:]:
    race = fastf1.get_session(year, year_schedule['Location'][round], 'R')
    race.load()
    dataset = race.laps

     # Aggregate additional data to the dataset
    df_to_append = pd.DataFrame(columns=['avg_speed', 'max_speed', 'min_speed', 'avg_throttle', 
                                     'full_throttle', 'full_brake', 'air_temp', 'track_temp', 
                                     'wind_dir', 'wind_speed'])
    
    for index in range(len(dataset)):
        lap = (dataset.iloc[index])
        # get the telemetry data for the lap via convenient method
        telemetry = lap.get_car_data()
        # compute some metrics from the telemetry data
        # avg speed over the lap
        avg_speed = telemetry['Speed'].mean()
        # max speed over the lap
        max_speed = telemetry['Speed'].max()
        # min speed over the lap
        min_speed = telemetry['Speed'].min()
        # avg throttle % over the lap
        avg_throttle = telemetry['Throttle'].mean()
        # % of the lap spent at full throttle
        full_throttle = telemetry[telemetry['Throttle'] == 100].shape[0] / telemetry.shape[0]
        # % of the lap spent breaking
        full_brake = telemetry[telemetry['Brake'] == True].shape[0] / telemetry.shape[0]
    
        # get the weather data for the lap
        weather = lap.get_weather_data()
        # get some data from the weather data
        # air temperature
        air_temp = weather['AirTemp']
        # track temperature
        track_temp = weather['TrackTemp']
        # wind direction
        wind_dir = weather['WindDirection']
        # wind speed
        wind_speed = weather['WindSpeed']

        # create a new dataframe with the new data
        df_to_append.loc[index] = [avg_speed, max_speed, min_speed, avg_throttle, full_throttle, full_brake, air_temp, track_temp, wind_dir, wind_speed]

    #add the new data to the current lap data
    dataset = pd.concat([dataset, df_to_append], axis=1)
    print('after: \n', dataset)

    # Clean up the dataset before dimensionality reduction
    # Remove rows that correspond to laps where the driver pitted, beacause they are obvious outliers
    rows_to_remove = dataset.index[dataset['PitInTime'].notnull()].tolist()
    dataset = dataset.drop(rows_to_remove, axis = 0)
    rows_to_remove = dataset.index[dataset['PitOutTime'].notnull()].tolist()
    dataset = dataset.drop(rows_to_remove, axis = 0)
    
    cols_to_remove =['Time', 'DriverNumber','PitOutTime','PitInTime', 'Sector1SessionTime', 
                        'Sector2SessionTime', 'Sector3SessionTime', 'IsPersonalBest',
                        'FreshTyre','LapStartTime','LapStartDate', 'Deleted',
                        'DeletedReason','FastF1Generated','IsAccurate']
    dataset = dataset.drop(cols_to_remove, axis = 1)
    dataset['TrackStatus'] = dataset['TrackStatus'].astype('float') #I hate this
    #drop rows where track status is not 1, i.e. the lap is not a 'racing' lap
    rows_to_remove = dataset.index[dataset['TrackStatus'] != 1].tolist()
    dataset = dataset.drop(rows_to_remove, axis = 0)
    #drop rows with missing data because i cant think of something better
    dataset = dataset.dropna(axis=0)
    #drop trackStatus as well after i used i to remove the rows i wanted
    dataset = dataset.drop('TrackStatus', axis = 1)
    #Convert timedelta64 to seconds
    cols_to_change = ['LapTime','Sector1Time',
          'Sector2Time', 'Sector3Time']
    for col in cols_to_change:
        dataset[col] = dataset[col].dt.total_seconds()
    dataset = dataset.reset_index()
    print(dataset.info())

    # MDS
    cols = ['Sector1Time','Sector2Time', 
            'Sector3Time', 'SpeedI1', 'SpeedI2',
            'SpeedFL', 'SpeedST', 'TyreLife', 'avg_speed', 
            'max_speed', 'min_speed', 'avg_throttle', 
            'full_throttle', 'full_brake', 'air_temp', 
            'track_temp', 'wind_dir', 'wind_speed']

    scaler= StandardScaler()
    scaled = scaler.fit_transform(dataset[cols])

    mds = MDS(n_components = 2, random_state=16)
    mds_results = mds.fit_transform(dataset[cols])
    mds_df = pd.DataFrame(data = mds_results, columns = ['dim1','dim2'])

    #KMeans
    kmeans = KMeans(n_clusters = 3)
    clusters = kmeans.fit(mds_df)
    labels = kmeans.labels_
    clusters = pd.concat([mds_df, pd.DataFrame({'kmeans_clusters':labels})], axis=1)
    clusters.columns=['dim1','dim2','cluster_id']
    result_df = pd.concat([dataset, clusters], axis=1)
       
    print(f'\n\n\n {year_schedule["Location"][round]} results_df: \n', result_df)

    dirname = '../data/'+str(round)+'_'+year_schedule['Location'][round]
    result_df.to_csv(dirname+'/'+'MDS.csv')

core           INFO 	Loading data for Bahrain Grand Prix - Race [v3.2.0]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for lap_count
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['1', '11', '14', '55', '44', '18', '63', '77', '10', '23', '22', '2', '20', '21', '27', '24', '4', '31', '16', '81']


after: 
                        Time Driver DriverNumber                LapTime  \
0    0 days 01:04:15.961000    VER            1 0 days 00:01:39.019000   
1    0 days 01:05:53.935000    VER            1 0 days 00:01:37.974000   
2    0 days 01:07:31.941000    VER            1 0 days 00:01:38.006000   
3    0 days 01:09:09.917000    VER            1 0 days 00:01:37.976000   
4    0 days 01:10:47.952000    VER            1 0 days 00:01:38.035000   
...                     ...    ...          ...                    ...   
1051 0 days 01:17:53.364000    PIA           81 0 days 00:01:41.534000   
1052 0 days 01:19:34.948000    PIA           81 0 days 00:01:41.584000   
1053 0 days 01:21:16.300000    PIA           81 0 days 00:01:41.352000   
1054 0 days 01:22:57.456000    PIA           81 0 days 00:01:41.156000   
1055 0 days 01:24:59.527000    PIA           81 0 days 00:02:02.071000   

      LapNumber  Stint PitOutTime              PitInTime  \
0           1.0    1.0        NaT         

core           INFO 	Loading data for Saudi Arabian Grand Prix - Race [v3.2.0]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info





 Sakhir results_df: 
      index Driver  LapTime  LapNumber  Stint  Sector1Time  Sector2Time  \
0        3    VER   97.976        4.0    1.0       31.271       42.642   
1        5    VER   97.986        6.0    1.0       31.341       42.632   
2        6    VER   98.021        7.0    1.0       31.290       42.763   
3       11    VER   98.591       12.0    1.0       31.408       42.951   
4       12    VER   98.482       13.0    1.0       31.389       42.907   
..     ...    ...      ...        ...    ...          ...          ...   
552   1050    PIA  101.295        8.0    1.0       31.665       44.590   
553   1051    PIA  101.534        9.0    1.0       31.989       44.548   
554   1052    PIA  101.584       10.0    1.0       31.779       44.617   
555   1053    PIA  101.352       11.0    1.0       31.894       44.557   
556   1054    PIA  101.156       12.0    1.0       32.100       44.235   

     Sector3Time  SpeedI1  SpeedI2  ...  avg_throttle  full_throttle  \
0         24.0

req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for lap_count
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['11', '1', '14', '63', '44', '55', '16', '31', '10', '20', '22', '27', '24', '21', '81', '2', '4', '77', '23', '18']


...




 Mexico City results_df: 
      index Driver  LapTime  LapNumber  Stint  Sector1Time  Sector2Time  \
0        2    VER   83.573        3.0    1.0       29.517       32.562   
1        3    VER   83.708        4.0    1.0       29.481       32.702   
2        5    VER   83.339        6.0    1.0       29.470       32.504   
3        6    VER   83.453        7.0    1.0       29.454       32.574   
4       10    VER   83.626       11.0    1.0       29.570       32.439   
..     ...    ...      ...        ...    ...          ...          ...   
830   1274    PIA   82.977       64.0    3.0       28.967       32.339   
831   1276    PIA   83.133       66.0    3.0       29.018       32.523   
832   1278    PIA   83.362       68.0    3.0       29.140       32.508   
833   1279    PIA   83.168       69.0    3.0       29.060       32.546   
834   1280    PIA   82.980       70.0    3.0       29.112       32.334   

     Sector3Time  SpeedI1  SpeedI2  ...  avg_throttle  full_throttle  \
0        

req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for lap_count
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['1', '4', '14', '11', '18', '55', '10', '44', '22', '31', '2', '27', '3', '81', '63', '77', '24', '20', '23', '16']


after: 
                        Time Driver DriverNumber                LapTime  \
0    0 days 01:04:23.001000    VER            1 0 days 00:01:59.857000   
1    0 days 01:06:46.357000    VER            1 0 days 00:02:23.356000   
2    0 days 01:32:11.479000    VER            1                    NaT   
3    0 days 01:34:24.149000    VER            1 0 days 00:02:12.670000   
4    0 days 01:35:39.894000    VER            1 0 days 00:01:15.745000   
...                     ...    ...          ...                    ...   
1104 0 days 01:56:43.133000    ZHO           24 0 days 00:01:16.767000   
1105 0 days 01:58:13.479000    ZHO           24 0 days 00:01:30.346000   
1106 0 days 01:04:23.001000    MAG           20                    NaT   
1107 0 days 01:04:23.001000    ALB           23                    NaT   
1108 0 days 01:04:23.001000    LEC           16                    NaT   

      LapNumber  Stint             PitOutTime              PitInTime  \
0           1.0    1.0        

logger      WARNING 	Failed to load schedule from FastF1 backend!
req            INFO 	Using cached data for season_schedule
core           INFO 	Loading data for Las Vegas Grand Prix - Race [v3.2.0]
req            INFO 	No cached data found for session_info. Loading data...
_api           INFO 	Fetching session info data...
logger      WARNING 	Failed to load session info data!
req            INFO 	No cached data found for driver_info. Loading data...
_api           INFO 	Fetching driver list...
core        WARNING 	Failed to load extended driver information!


KeyError: 'DriverNumber'

# Tests

In [5]:
print('before:\n', laps_og)

laps = laps_og.copy()
df_to_append = pd.DataFrame(columns=['avg_speed', 'max_speed', 'min_speed', 'avg_throttle', 
                                     'full_throttle', 'full_brake', 'air_temp', 'track_temp', 
                                     'wind_dir', 'wind_speed'])

for index in range(len(laps)):
    lap = (laps.iloc[index])
    # get the telemetry data for the lap via convenient method
    telemetry = lap.get_car_data()
    # compute some metrics from the telemetry data
    # avg speed over the lap
    avg_speed = telemetry['Speed'].mean()
    # max speed over the lap
    max_speed = telemetry['Speed'].max()
    # min speed over the lap
    min_speed = telemetry['Speed'].min()
    # avg throttle % over the lap
    avg_throttle = telemetry['Throttle'].mean()
    # % of the lap spent at full throttle
    full_throttle = telemetry[telemetry['Throttle'] == 100].shape[0] / telemetry.shape[0]
    # % of the lap spent breaking
    full_brake = telemetry[telemetry['Brake'] == True].shape[0] / telemetry.shape[0]
    
    # get the weather data for the lap
    weather = lap.get_weather_data()
    # get some data from the weather data
    # air temperature
    air_temp = weather['AirTemp']
    # track temperature
    track_temp = weather['TrackTemp']
    # wind direction
    wind_dir = weather['WindDirection']
    # wind speed
    wind_speed = weather['WindSpeed']

    # create a new dataframe with the new data
    df_to_append.loc[index] = [avg_speed, max_speed, min_speed, avg_throttle, full_throttle, full_brake, air_temp, track_temp, wind_dir, wind_speed]
    
    #add the new data to the current lap data
    #laps.iloc[index] = pd.concat([lap, new_data])
laps = pd.concat([laps, df_to_append], axis=1)
print('after: \n', laps)

#The data is now aggregated, time for tSNE


before:
                        Time Driver DriverNumber                LapTime  \
0    0 days 01:04:15.961000    VER            1 0 days 00:01:39.019000   
1    0 days 01:05:53.935000    VER            1 0 days 00:01:37.974000   
2    0 days 01:07:31.941000    VER            1 0 days 00:01:38.006000   
3    0 days 01:09:09.917000    VER            1 0 days 00:01:37.976000   
4    0 days 01:10:47.952000    VER            1 0 days 00:01:38.035000   
...                     ...    ...          ...                    ...   
1051 0 days 01:17:53.364000    PIA           81 0 days 00:01:41.534000   
1052 0 days 01:19:34.948000    PIA           81 0 days 00:01:41.584000   
1053 0 days 01:21:16.300000    PIA           81 0 days 00:01:41.352000   
1054 0 days 01:22:57.456000    PIA           81 0 days 00:01:41.156000   
1055 0 days 01:24:59.527000    PIA           81 0 days 00:02:02.071000   

      LapNumber  Stint PitOutTime              PitInTime  \
0           1.0    1.0        NaT         

In [20]:
print(laps.info())

import numpy as np
import pandas as pd
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans
from sklearn.discriminant_analysis import StandardScaler

dataset = laps
rows_to_remove = dataset.index[dataset['PitInTime'].notnull()].tolist()
dataset = dataset.drop(rows_to_remove, axis = 0)
rows_to_remove = dataset.index[dataset['PitOutTime'].notnull()].tolist()
dataset = dataset.drop(rows_to_remove, axis = 0)
cols_to_remove =['Time', 'DriverNumber','PitOutTime','PitInTime', 'Sector1SessionTime', 
   'Sector2SessionTime', 'Sector3SessionTime', 'IsPersonalBest',
   'FreshTyre','LapStartTime','LapStartDate', 'Deleted',
   'DeletedReason','FastF1Generated','IsAccurate']
dataset = dataset.drop(cols_to_remove, axis = 1)
dataset['TrackStatus'] = dataset['TrackStatus'].astype('float') #I hate this
#drop rows where track status is not 1, i.e. the lap is not a 'racing' lap
rows_to_remove = dataset.index[dataset['TrackStatus'] != 1].tolist()
dataset = dataset.drop(rows_to_remove, axis = 0)

#drop rows with missing data
dataset = dataset.dropna(axis=0)
#drop trackStatus as well after i used it to remove the rows i wanted
dataset = dataset.drop('TrackStatus', axis = 1)
#Convert timedelta64 to seconds
cols_to_change = ['LapTime','Sector1Time',
'Sector2Time', 'Sector3Time']
for col in cols_to_change:
    dataset[col] = dataset[col].dt.total_seconds()
dataset = dataset.reset_index()
print(dataset.info())

cols = ['LapTime', 'Sector1Time',
'Sector2Time', 'Sector3Time', 'SpeedI1', 'SpeedI2',
'SpeedFL', 'SpeedST', 'TyreLife', 'avg_speed', 
'max_speed', 'min_speed', 'avg_throttle', 'full_throttle', 
'full_brake', 'air_temp', 'track_temp', 'wind_dir', 'wind_speed']

print(dataset[cols].info())

scaler= StandardScaler()
scaler.fit(dataset[cols])
scaled = scaler.fit_transform(dataset[cols])
print(scaled)
RS = 16
tsne = TSNE(n_components=2, learning_rate = 100, n_iter=5000,
     n_iter_without_progress=500, random_state=RS)
tsne_results = tsne.fit_transform(scaled)
tsne_df = pd.DataFrame(data = tsne_results, columns=['dim1','dim2'])
print(tsne_df)

#KMeans
k_means_tsne = KMeans(n_clusters=3, n_init=100, max_iter=400, init="k-means++", random_state=RS).fit(tsne_df)
labels_tsne = k_means_tsne.labels_
clusters_tsne = pd.concat([tsne_df, pd.DataFrame({'tsne_clusters':labels_tsne})], axis=1)
clusters_tsne.columns=['dim1','dim2','tsne_clusters_id']
print(clusters_tsne)
print(clusters_tsne['tsne_clusters_id'].value_counts())

result_df = pd.concat([dataset, clusters_tsne], axis=1)
print(result_df)

dirname = '../data/1_Sakhir'
result_df.to_csv(dirname+'/'+'tSNE_2.csv')

<class 'fastf1.core.Laps'>
RangeIndex: 1056 entries, 0 to 1055
Data columns (total 41 columns):
 #   Column              Non-Null Count  Dtype          
---  ------              --------------  -----          
 0   Time                1056 non-null   timedelta64[ns]
 1   Driver              1056 non-null   object         
 2   DriverNumber        1056 non-null   object         
 3   LapTime             1055 non-null   timedelta64[ns]
 4   LapNumber           1056 non-null   float64        
 5   Stint               1056 non-null   float64        
 6   PitOutTime          50 non-null     timedelta64[ns]
 7   PitInTime           52 non-null     timedelta64[ns]
 8   Sector1Time         1035 non-null   timedelta64[ns]
 9   Sector2Time         1055 non-null   timedelta64[ns]
 10  Sector3Time         1055 non-null   timedelta64[ns]
 11  Sector1SessionTime  1031 non-null   timedelta64[ns]
 12  Sector2SessionTime  1055 non-null   timedelta64[ns]
 13  Sector3SessionTime  1055 non-null   timedelt